In [ ]:
import pandas as pd

# file
df_area3_csv = pd.read_csv("DATA/Study_AREA3.csv")
df_area3_csv["Date"] = pd.to_datetime(df_area3_csv["Date"])
if "Study_Area" not in df_area3_csv.columns:
    df_area3_csv["Study_Area"] = 3

# Define time difference feature first to ensure it's present
def add_time_diff_days_global(df):
    if "E.coli" in df.columns:
        ecoli_dates = df["Date"].where(df["E.coli"].notna())
        df["time_diff_days_global"] = ecoli_dates.ffill().diff().dt.days.fillna(0)
    return df

# Apply full feature engineering
def full_feature_engineering(df):
    df = df.copy()
    df["month"] = df["Date"].dt.month
    df["season"] = (df["month"] % 12 // 3 + 1)

    if "Precipitation" in df.columns:
        monthly_avg = df.groupby("month")["Precipitation"].transform("mean")
        seasonal_avg = df.groupby("season")["Precipitation"].transform("mean")
        df["rainfall_anomaly"] = df["Precipitation"] - monthly_avg
        df["seasonal_rainfall_deviation"] = df["Precipitation"] - seasonal_avg

        for days in [2, 3, 4, 5, 6, 7, 30]:

            df[f"cum_rain_{days}_days"] = df.groupby("Study_Area")["Precipitation"].rolling(
                window=days, min_periods=1).sum().reset_index(level=0, drop=True)
            df[f"cum_rain_{days}_days_lag1"] = df.groupby("Study_Area")["Precipitation"].shift(1).rolling(
                window=days, min_periods=1).sum().reset_index(level=0, drop=True)


        df["max_rain_7_days"] = df.groupby("Study_Area")["Precipitation"].rolling(
            window=7, min_periods=1).max().reset_index(level=0, drop=True)
        df["max_rain_7_days_lag1"] = df.groupby("Study_Area")["Precipitation"].shift(1).rolling(
            window=7, min_periods=1).max().reset_index(level=0, drop=True)


        df["percent_rainy_days"] = df["Precipitation"].gt(0).astype(int).groupby(
            df["Study_Area"]).rolling(30, min_periods=1).mean().reset_index(level=0, drop=True)


        df["rainfall_variability_30_days"] = df.groupby("Study_Area")["Precipitation"].rolling(
            window=30, min_periods=1).std().reset_index(level=0, drop=True)


        df["ema_rain_7_days"] = df.groupby("Study_Area")["Precipitation"].ewm(
            span=7, adjust=False).mean().reset_index(level=0, drop=True)

        df["percent_rainy_days_lag1"] = df["percent_rainy_days"].shift(1)
        df["rainfall_variability_30_days_lag1"] = df["rainfall_variability_30_days"].shift(1)
        df["ema_rain_7_days_lag1"] = df["ema_rain_7_days"].shift(1)
        df["rainfall_anomaly_lag1"] = df["rainfall_anomaly"].shift(1)
        df["seasonal_rainfall_deviation_lag1"] = df["seasonal_rainfall_deviation"].shift(1)

        for lag in [1, 2, 3]:
            df[f"Precipitation_lag{lag}"] = df.groupby("Study_Area")["Precipitation"].shift(lag)

    df["dry_days"] = (df["Precipitation"] == 0).astype(int).groupby(
        df["Precipitation"].ne(df["Precipitation"].shift()).cumsum()).cumcount()
    df["wet_days"] = (df["Precipitation"] > 0).astype(int).groupby(
        df["Precipitation"].eq(0).cumsum()).cumcount()
    df["Antecedent_Wet_Period"] = (df["Precipitation"] > 0).astype(int).groupby(
        (df["Precipitation"] == 0).cumsum()).cumcount()
    df["antecedent_dry_period"] = (df["Precipitation"] <= 1).astype(int).groupby(
        (df["Precipitation"] > 5).cumsum()).cumcount()
    df["dry_days_lag1"] = df["dry_days"].shift(1)
    df["wet_days_lag1"] = df["wet_days"].shift(1)
    df["Antecedent_Wet_Period_lag1"] = df["Antecedent_Wet_Period"].shift(1)
    df["antecedent_dry_period_lag1"] = df["antecedent_dry_period"].shift(1)


    vars_to_lag = [
        "Streamflow", "Gage Height", "Upstream Tributary Streamflow",
        "Water Temperature", "Turbidity", "Evapotranspiration",
        "Specific Humidity", "Solar Radiation", "Wind Speed",
        "Air Temperature", "Relative Humidity"
    ]
    for var in vars_to_lag:
        if var in df.columns:
            for lag in [1, 2, 3]:
                df[f"{var}_lag{lag}"] = df.groupby("Study_Area")[var].shift(lag)

    return add_time_diff_days_global(df)

# Apply the complete processing
df_area3_final = full_feature_engineering(df_area3_csv)
df_area3_final["Date"] = df_area3_final["Date"].dt.strftime("%m/%d/%Y")

# Save again with updated time_diff_days_global
final_area3_csv_path_updated = "Study.xlsx"
df_area3_final.to_excel(final_area3_csv_path_updated, index=False)

final_area3_csv_path_updated

'Study.xlsx'